In [34]:
print("테스트")

테스트


# 데이터/패키지 로딩

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns

train = pd.read_csv("data/titanic/train.csv", index_col = "PassengerId")
test = pd.read_csv("data/titanic/test.csv", index_col = "PassengerId")

print(train.shape, test.shape)

train.head()

(891, 11) (418, 10)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 데이터 전처리

## (1) NaN 처리

In [36]:
#train.columns
#train[['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']].describe()
train.info() 

#Age, Cabin, Embarked => Cabin은 너무 빈값이 많아서 일단 제외

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [37]:
test[['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked']].describe() #Age, Fare
test.info() 
# Age, Fare, Cabin => Cabin은 빈값이 많아서 일단 제외

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
Pclass      418 non-null int64
Name        418 non-null object
Sex         418 non-null object
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Ticket      418 non-null object
Fare        417 non-null float64
Cabin       91 non-null object
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


In [38]:
train.loc[train["Age"].isnull()].shape

(177, 11)

In [46]:
train["Age_encode"] = train["Age"]
train.loc[train["Age"].isnull(), "Age_encode"] = train["Age"].mean()
print(train.loc[train["Age_encode"].isnull()].shape)
#train.loc[train["Age"].isnull()]
#train.head()

test["Age_encode"] = test["Age"]
test.loc[test["Age"].isnull(), "Age_encode"] = test["Age"].mean()
print(test.loc[test["Age_encode"].isnull()].shape)

(0, 12)
(0, 11)


In [52]:
train["Fare_encode"] = train["Fare"]
train.loc[train["Fare"].isnull(), "Fare_encode"] = train["Fare"].mean()
print(train.loc[train["Fare_encode"].isnull()].shape)
#train.loc[train["Age"].isnull()]
#train.head()

test["Fare_encode"] = test["Fare"]
test.loc[test["Fare"].isnull(), "Fare_encode"] = test["Fare"].mean()
print(test.loc[test["Fare_encode"].isnull()].shape)

(0, 14)
(0, 13)


# Sex, Embarked Encoding

In [53]:
train["Sex_encode"] = train["Sex"]
train.loc[train["Sex"] == "male", "Sex_encode"] = 0
train.loc[train["Sex"] == "female", "Sex_encode"] = 1
# train[["Sex_encode", "Sex"]]

test["Sex_encode"] = test["Sex"]
test.loc[test["Sex"] == "male", "Sex_encode"] = 0
test.loc[test["Sex"] == "female", "Sex_encode"] = 1

In [88]:
category_feature = ["Survived", "Pclass", "Sex_encode"]

# print(train.info(), test.info())

for var in category_feature:
    train[var] = train[var].astype("category")
    
category_feature_test = ["Pclass", "Sex_encode"]

for var in category_feature_test:    
    test[var] = test[var].astype("category")

# Feature 정의

In [89]:
train.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'Age_encode', 'Sex_encode', 'Fare_encode'],
      dtype='object')

In [90]:
feature_names = ['Pclass', 'SibSp', 'Parch', 'Fare_encode', 'Age_encode', 'Sex_encode']
label_name = 'Survived'

# 모델로딩

In [91]:
X_train = train[feature_names]
X_test = test[feature_names]
y_train = train[label_name]

print(X_train.shape, X_test.shape, y_train.shape)

(891, 6) (418, 6) (891,)


In [102]:
from sklearn.ensemble import GradientBoostingClassifier
GradientBoostingClassifier()

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [135]:
gbm = GradientBoostingClassifier(n_estimators = 4000, 
                                 max_depth = 10, 
                                 random_state = 79);

gbm.fit(X_train, y_train)
predictions = gbm.predict(X_test)

predictions.shape

(418,)

In [136]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

In [137]:
train_score = gbm.train_score_
train_score_df = pd.DataFrame(train_score)
train_score_df.head()
train_score_df.index
#sns.distplot(train_score)

RangeIndex(start=0, stop=4000, step=1)

fig, (ax1, ax2) = plt.subplots(ncols = 2)
fig.set_size_inches(16, 5)

sns.barplot(data = y_train, ax = ax1, bins = 50)
ax1.set(title = "train")
sns.barplot(data = predictions, ax = ax2, bins = 50)
ax2.set(title = "test")

# Feature Importance

In [138]:
print(list(zip(feature_names, gbm.feature_importances_)))

feature_df = list(zip(feature_names, gbm.feature_importances_))
feature_df = pd.DataFrame(feature_df)

[('Pclass', -492001129952.9425), ('SibSp', 246546946980.27628), ('Parch', 6930176920.454164), ('Fare_encode', 181363895424.26633), ('Age_encode', 57165667769.119095), ('Sex_encode', -5557140.447326013)]


In [139]:
feature_df.columns = feature_df.columns.astype(str)
# RangeIndex(start=0, stop=2, step=1)로 되어 있던 것은 문자로 변경

feature_df.columns = ["feature", "importance"]
# 컬럼명 변경, 아래처럼 하는 것은 안됨
# feature_df.rename(columns={'0':"feature", '1':"feature_importance"})

feature_df.sort_values(by=['importance', 'feature'], ascending=[False, True], inplace=True)
# importance 먼저 정렬, 그리고 importance 정렬

feature_df

,feature,importance
1,SibSp,2.465469e+11
3,Fare_encode,1.813639e+11
4,Age_encode,5.716567e+10
2,Parch,6.930177e+09
5,Sex_encode,-5.557140e+06
0,Pclass,-4.920011e+11


In [140]:
feature_df["feature"].values

array(['SibSp', 'Fare_encode', 'Age_encode', 'Parch', 'Sex_encode',
       'Pclass'], dtype=object)

# Submission

In [141]:
submission = pd.read_csv("data/titanic/gender_submission.csv", index_col = "PassengerId")
submission.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [142]:
predictions.shape

(418,)

In [143]:
submission["Survived"] = predictions
submission.head()

,Survived
PassengerId,
892,0
893,0
894,1
895,1
896,1


In [144]:
submission.to_csv("data/titanic/submission_20180911.csv")